<a href="https://colab.research.google.com/github/reeaarana/Research-Paper-Recommendation-System/blob/main/nlp_rs_proj.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# --- Install dependencies ---
!pip install -q kagglehub pymupdf sentence-transformers scikit-learn pandas faiss-cpu

import kagglehub
import pandas as pd
import numpy as np
import fitz  # PyMuPDF
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity
import faiss
from google.colab import files


In [6]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import numpy as np

# Generate some sample data
X = np.random.rand(100, 1) * 10
y = 2 * X + 1 + np.random.randn(100, 1)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create and train the model
model = LinearRegression()
model.fit(X_train, y_train)

# Evaluate the model
score = model.score(X_test, y_test)
print(f"Model R-squared score: {score:.4f}")

# Make predictions
predictions = model.predict(X_test)
print("\nSample predictions:")
for i in range(5):
    print(f"Input: {X_test[i][0]:.2f}, Actual: {y_test[i][0]:.2f}, Predicted: {predictions[i][0]:.2f}")

Model R-squared score: 0.9646

Sample predictions:
Input: 7.46, Actual: 17.91, Predicted: 15.90
Input: 4.38, Actual: 9.20, Predicted: 9.66
Input: 6.20, Actual: 15.45, Predicted: 13.36
Input: 4.18, Actual: 9.71, Predicted: 9.27
Input: 8.57, Actual: 16.90, Predicted: 18.15


In [10]:
# -----------------------------
# Download ArXiv dataset from Kaggle (via kagglehub) and load abstracts-only
# -----------------------------
# Paste this cell and run it BEFORE the dataset loader cell (or replace your current download block with this).

# 1) Install kagglehub if needed (works in Colab)
try:
    import kagglehub
except Exception:
    !pip install -q kagglehub
    import kagglehub

import os
from pathlib import Path

print("Using kagglehub to download Cornell-University/arxiv dataset (same approach as your original notebook).")

# Optional: you can pass version=<int> to dataset_download for a particular snapshot version
try:
    ds_path = kagglehub.dataset_download("Cornell-University/arxiv")  # returns local path to extracted files
    print("Downloaded/extracted dataset to:", ds_path)
except Exception as e:
    # Provide helpful fallback if download fails
    raise RuntimeError(
        "kagglehub.dataset_download(...) failed. Make sure your environment has internet access and Kaggle credentials if required. "
        "Original error: " + str(e)
    )

# 2) Locate the expected JSON file inside the dataset dir
ds_path = Path(ds_path)
candidates = [
    ds_path / "arxiv-metadata-oai-snapshot.json",
    ds_path / "arxiv-metadata-oai-snapshot.json.gz",
    ds_path / "arxiv.jsonl",
    ds_path / "arxiv.jsonl.gz",
    # add any other plausible names
]
json_path = None
for c in candidates:
    if c.exists():
        json_path = str(c)
        break

# If not found, try a light scan of the dataset dir for any .json or .jsonl file
if json_path is None:
    for p in ds_path.rglob("*.json*"):
        # prefer files with 'arxiv' in name
        if "arxiv" in p.name.lower():
            json_path = str(p)
            break
    if json_path is None:
        # try any .jsonl or .json
        p_any = next(ds_path.rglob("*.*json*"), None)
        if p_any:
            json_path = str(p_any)

if json_path is None:
    raise FileNotFoundError(f"Could not find an arxiv JSON/JSONL file inside {ds_path}. Listing files for debugging:\n{list(ds_path.iterdir())[:50]}")

print("Found dataset file:", json_path)

# 3) Set JSONL_PATH so your loader picks it up, and call your loader (abstract-only)
JSONL_PATH = json_path  # loader uses this global if present
# if you have the loader function already defined as load_arxiv_jsonl_abstracts, call it now:
try:
    # tune limit as you want (e.g., 50000). Set to None to load all (may be large).
    LIMIT = 50000
    df = load_arxiv_jsonl_abstracts(JSONL_PATH, limit=LIMIT)
    print("Loaded df shape:", df.shape)
    display(df.head(3))
except NameError:
    print("The loader function 'load_arxiv_jsonl_abstracts' is not defined in the notebook. "
          "Please run the loader cell (the abstracts-only loader) after this cell, or paste it above this cell.")


Using kagglehub to download Cornell-University/arxiv dataset (same approach as your original notebook).


100%|██████████| 1.52G/1.52G [00:06<00:00, 245MB/s]

Extracting files...


Downloaded/extracted dataset to: /root/.cache/kagglehub/datasets/Cornell-University/arxiv/versions/257
Found dataset file: /root/.cache/kagglehub/datasets/Cornell-University/arxiv/versions/257/arxiv-metadata-oai-snapshot.json


Reading JSONL:   0%|          | 0/2866787 [00:00<?, ?it/s]

The loader function 'load_arxiv_jsonl_abstracts' is not defined in the notebook. Please run the loader cell (the abstracts-only loader) after this cell, or paste it above this cell.


In [11]:
# -----------------------------
# Define abstracts-only loader (cleaning + load) and run it now
# -----------------------------
import os, json, re, gzip
from tqdm.auto import tqdm
import pandas as pd
from typing import Optional

# --- cleaning helper (keeps text useful for embeddings) ---
def clean_abstract_text(s: str) -> str:
    """Light cleaning to improve embedding quality: remove math, refs, URLs, extra whitespace."""
    if not s:
        return ""
    # Remove LaTeX math (inline and display)
    s = re.sub(r"\$[^$]*\$", " ", s)
    s = re.sub(r"\\\[.*?\\\]", " ", s, flags=re.DOTALL)
    s = re.sub(r"\\\(.+?\\\)", " ", s, flags=re.DOTALL)
    # Remove URLs
    s = re.sub(r"https?://\S+|www\.\S+", " ", s)
    # Remove common reference labels like [1], (1)
    s = re.sub(r"\[\s*\d+(?:,\s*\d+)*\s*\]", " ", s)
    s = re.sub(r"\(\s*\d+(?:,\s*\d+)*\s*\)", " ", s)
    # Remove trailing References/Acknowledgements blocks if accidentally included
    s = re.sub(r"(?is)\breferences\b.*", " ", s)
    s = re.sub(r"(?is)\backnowledg", " ", s)
    # Normalize whitespace and trim
    s = re.sub(r"\s+", " ", s).strip()
    return s[:4000]

# --- loader (supports .gz) ---
def load_arxiv_jsonl_abstracts(path: str, limit: Optional[int] = None, skip: int = 0) -> pd.DataFrame:
    path = str(path)
    if not os.path.exists(path):
        raise FileNotFoundError(f"Dataset not found at: {path}")
    rows = []
    open_fn = gzip.open if path.endswith(".gz") else open
    total_read = 0
    total_kept = 0

    # optional small progress-friendly wrapper
    with open_fn(path, "rt", encoding="utf-8", errors="ignore") as fh:
        iterator = enumerate(fh)
        iterator = tqdm(iterator, desc="Reading JSONL", unit="lines")
        for i, line in iterator:
            if i < skip:
                continue
            if limit is not None and (total_read - skip) >= limit:
                break
            total_read += 1
            line = line.strip()
            if not line:
                continue
            try:
                obj = json.loads(line)
            except Exception:
                continue
            title = (obj.get("title") or "").strip()
            abstract = (obj.get("abstract") or "").strip()
            if not abstract:
                continue
            abstract_clean = clean_abstract_text(abstract)
            if not abstract_clean:
                continue
            rows.append({"title": title, "abstract": abstract_clean, "text": abstract_clean})
            total_kept += 1

    df_local = pd.DataFrame(rows)
    print(f"Loaded {len(df_local):,} papers (abstracts only). Read {total_read:,} lines, kept {total_kept:,}.")
    return df_local

# --- Run loader using JSONL_PATH set by your kagglehub cell ---
# JSONL_PATH should already be set (the kagglehub cell printed it). If not, set it manually here.
try:
    JSONL_PATH  # noqa: F821
except NameError:
    raise RuntimeError("JSONL_PATH is not defined. Re-run the kagglehub download cell or set JSONL_PATH to the dataset file path.")

# Tune this limit (None to load all; loading all ~2.8M lines may be heavy)
LIMIT = 50000

# Load into df
df = load_arxiv_jsonl_abstracts(JSONL_PATH, limit=LIMIT)
print("Dataframe shape:", df.shape)
display(df.head(3))


Reading JSONL: 0lines [00:00, ?lines/s]

Loaded 50,000 papers (abstracts only). Read 50,000 lines, kept 50,000.
Dataframe shape: (50000, 3)


,title,abstract,text
0,Calculation of prompt diphoton production cros...,A fully differential calculation in perturbati...,A fully differential calculation in perturbati...
1,Sparsity-certifying Graph Decompositions,"We describe a new algorithm, the -pebble game ...","We describe a new algorithm, the -pebble game ..."
2,The evolution of the Earth-Moon system based o...,The evolution of Earth-Moon system is describe...,The evolution of Earth-Moon system is describe...


In [13]:
# -----------------------------
# Full replacement: Embeddings + FAISS (self-contained)
# -----------------------------
import os
import logging
from pathlib import Path
import numpy as np
from sentence_transformers import SentenceTransformer
import faiss

# set up logger (so this cell is self-contained)
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger("embeddings_faiss")

# Paths and model config
ARTIFACT_DIR = Path("/content")
EMB_PATH = ARTIFACT_DIR / "embeddings_abstracts.npy"
FAISS_INDEX_PATH = ARTIFACT_DIR / "faiss_abstracts_hnsw.idx"
EMBED_MODEL_NAME = "all-mpnet-base-v2"  # higher-quality encoder
BATCH_SIZE = 256

# Safety checks: ensure `df` exists and has 'text'
if 'df' not in globals():
    raise RuntimeError("DataFrame 'df' not found in globals. Run the abstracts loader cell first.")
if 'text' not in df.columns:
    raise RuntimeError("DataFrame 'df' does not contain a 'text' column (abstracts). Ensure loader produced df['text'].")

def build_or_load_embeddings_and_index(texts, emb_path=EMB_PATH, index_path=FAISS_INDEX_PATH, model_name=EMBED_MODEL_NAME):
    # 1) embeddings cache load/build
    if emb_path.exists():
        logger.info("Loading embeddings from %s", emb_path)
        embeddings = np.load(str(emb_path))
    else:
        logger.info("Encoding %d abstracts with %s (batch_size=%d)...", len(texts), model_name, BATCH_SIZE)
        model = SentenceTransformer(model_name)
        embs = []
        for i in range(0, len(texts), BATCH_SIZE):
            batch = texts[i:i+BATCH_SIZE]
            be = model.encode(batch, show_progress_bar=True, convert_to_numpy=True)
            embs.append(be)
        embeddings = np.vstack(embs).astype("float32")
        # Ensure artifact dir exists
        emb_path.parent.mkdir(parents=True, exist_ok=True)
        np.save(str(emb_path), embeddings)
        logger.info("Saved embeddings to %s", emb_path)

    # 2) build/load HNSW index on normalized vectors (cosine via inner product)
    dim = embeddings.shape[1]
    # normalize in-place copy for index operations
    faiss.normalize_L2(embeddings)
    if index_path.exists():
        logger.info("Loading FAISS (HNSW) index from %s", index_path)
        index = faiss.read_index(str(index_path))
    else:
        logger.info("Building FAISS HNSW index (IndexHNSWFlat) for dim=%d ...", dim)
        # Choose M ~ 32 for good accuracy; tune efConstruction if desired
        M = 32
        index = faiss.IndexHNSWFlat(dim, M, faiss.METRIC_INNER_PRODUCT)
        # set efConstruction for better accuracy at build time
        try:
            index.hnsw.efConstruction = 200
        except Exception:
            # some faiss builds expose hnsw differently; ignore if not available
            pass
        index.add(embeddings)
        # persist
        faiss.write_index(index, str(index_path))
        logger.info("Saved FAISS HNSW index to %s", index_path)

    return embeddings, index

# Run the builder / loader
texts = df["text"].astype(str).tolist()
embeddings, index = build_or_load_embeddings_and_index(texts)

# Basic sanity print
print("Embeddings shape:", embeddings.shape)
print("FAISS index ntotal:", index.ntotal)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

Batches:   0%|          | 0/8 [00:00<?, ?it/s]

Batches:   0%|          | 0/8 [00:00<?, ?it/s]

Batches:   0%|          | 0/8 [00:00<?, ?it/s]

Batches:   0%|          | 0/3 [00:00<?, ?it/s]

Embeddings shape: (50000, 768)
FAISS index ntotal: 50000


In [14]:
# -----------------------------
# Replace PDF extractor cell
# -----------------------------
import fitz  # already imported
import re

def _pymupdf_get_text(pdf_path: str) -> str:
    doc = fitz.open(pdf_path)
    pages = []
    for p in doc:
        pages.append(p.get_text())
    return "\n".join(pages)

# Reuse the earlier clean function for safety
def extract_abstract_from_pdf(pdf_path: str, use_ocr: bool = False) -> str:
    """
    Heuristic extraction: find the 'Abstract' header and take text until typical next section markers.
    If not found, fallback to first page paragraph (cleaned).
    """
    full_text = _pymupdf_get_text(pdf_path)
    if not full_text or not full_text.strip():
        if use_ocr:
            # optional: implement OCR fallback (requires pytesseract + tesseract binary)
            try:
                import pytesseract
                from PIL import Image
                text_parts = []
                doc = fitz.open(pdf_path)
                for page in doc:
                    pix = page.get_pixmap(dpi=200)
                    img = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)
                    text_parts.append(pytesseract.image_to_string(img))
                full_text = "\n".join(text_parts)
            except Exception:
                full_text = ""
    if not full_text:
        return ""

    # Normalize
    txt = full_text.replace("\r", "\n")
    # Find abstract header
    m = re.search(r"(?is)\babstract\b[:\s\-–—]*", txt)
    if m:
        start = m.end()
        # stop at common next section headers
        stop_patterns = [r"\n\s*1\.\s*Introduction\b", r"\n\s*Introduction\b", r"\n\s*I\.\s*Introduction\b",
                         r"\n\s*Keywords\b", r"\n\s*Index Terms\b", r"\n\s*References\b"]
        end = len(txt)
        for pat in stop_patterns:
            hit = re.search(pat, txt[start:], flags=re.IGNORECASE)
            if hit:
                end = start + hit.start()
                break
        abstract_raw = txt[start:end].strip()
        abstract_clean = clean_abstract_text(abstract_raw)
        if abstract_clean:
            return abstract_clean

    # fallback: take first page paragraph as abstract proxy
    first_page = txt.split("\f")[0] if "\f" in txt else txt.split("\n\n", 1)[0]
    first_page = re.sub(r"\s+", " ", first_page).strip()
    return clean_abstract_text(first_page)


In [15]:
# ============================================================
# Step 5: Hybrid Recommendation System
# ============================================================
def recommend_papers(pdf_path, top_k=10, shortlist_k=100):
    """
    Hybrid recommender:
    1. Use SVD/LSA to shortlist candidates
    2. Use embeddings + FAISS to rerank top papers
    """
    # Extract PDF text
    input_text = extract_text_from_pdf(pdf_path)

    # ----- Phase 1: SVD-based shortlist -----
    input_vec = vectorizer.transform([input_text])
    input_reduced = svd.transform(input_vec)
    sims_svd = cosine_similarity(input_reduced, X_reduced)[0]
    top_indices_svd = sims_svd.argsort()[-shortlist_k:][::-1]  # shortlist

    # ----- Phase 2: Embedding reranking -----
    input_emb = model.encode([input_text]).astype('float32')
    faiss.normalize_L2(input_emb)

    # Restrict FAISS to shortlisted candidates
    candidate_embs = embeddings[top_indices_svd]
    rerank_index = faiss.IndexFlatIP(dim)
    rerank_index.add(candidate_embs)
    D, I = rerank_index.search(input_emb, top_k)

    # Map FAISS results back to dataset indices
    reranked_indices = [top_indices_svd[i] for i in I[0][:top_k]]

    # ----- Output -----
    print(f"\n📄 Top {top_k} Recommended Papers (Hybrid: SVD shortlist + Embedding rerank):")
    for rank, (idx, score) in enumerate(zip(reranked_indices, D[0]), start=1):
        print(f"\n{rank}. 🔹 Title: {df.iloc[idx]['title']}")
        print(f"   Score: {score:.4f}")
        print(f"   Abstract: {df.iloc[idx]['abstract'][:300]}...")
        print("-"*120)

In [16]:
# -----------------------------
# Replace SVD/LSA cell
# -----------------------------
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
import joblib

# TUNE: increase max_features and n_components to capture more semantics in abstracts
MAX_FEATURES = 30000
N_COMPONENTS = 300

def fit_or_load_lsa_for_abstracts(df_texts, vect_path="/content/tfidf_abstracts.joblib", svd_path="/content/svd_abstracts.joblib"):
    if Path(vect_path).exists() and Path(svd_path).exists():
        vectorizer = joblib.load(vect_path)
        svd_model = joblib.load(svd_path)
        X_reduced = svd_model.transform(vectorizer.transform(df_texts))
        logger.info("Loaded saved vectorizer & SVD.")
        return vectorizer, svd_model, X_reduced

    logger.info("Fitting TF-IDF (max_features=%d) and SVD (n_components=%d) on abstracts...", MAX_FEATURES, N_COMPONENTS)
    vectorizer = TfidfVectorizer(stop_words='english', max_features=MAX_FEATURES, ngram_range=(1,2))
    X = vectorizer.fit_transform(df_texts)
    svd_model = TruncatedSVD(n_components=N_COMPONENTS, random_state=42)
    X_reduced = svd_model.fit_transform(X)

    joblib.dump(vectorizer, vect_path)
    joblib.dump(svd_model, svd_path)
    logger.info("Saved TF-IDF & SVD to disk.")
    return vectorizer, svd_model, X_reduced

vectorizer, svd_model, X_reduced = fit_or_load_lsa_for_abstracts(df["text"].tolist())
print("LSA matrix shape:", X_reduced.shape)


LSA matrix shape: (50000, 300)


In [17]:
# -----------------------------
# Replace recommend_papers cell
# -----------------------------
from sklearn.metrics.pairwise import cosine_similarity

def recommend_papers_from_pdf(pdf_path: str, top_k: int = 10, shortlist_k: int = 1000, use_ocr: bool = False):
    """
    End-to-end: extract abstract from PDF -> LSA shortlist -> dense rerank (FAISS HNSW) -> return top_k results.
    """
    query_abstract = extract_abstract_from_pdf(pdf_path, use_ocr=use_ocr)
    if not query_abstract:
        raise ValueError("Could not extract abstract from PDF. Try enabling OCR or check PDF type.")
    logger.info("Query abstract preview: %s", query_abstract[:200].replace("\n", " "))

    # LSA shortlist (fast coarse filter)
    q_vec = vectorizer.transform([query_abstract])
    q_red = svd_model.transform(q_vec) if 'svd_model' in globals() else svd.transform(q_vec)
    sims = cosine_similarity(q_red, X_reduced)[0]
    shortlist_idx = sims.argsort()[-shortlist_k:][::-1]

    # Dense rerank: encode query once with same embed model
    embed_model = SentenceTransformer(EMBED_MODEL_NAME)
    q_emb = embed_model.encode([query_abstract], convert_to_numpy=True).astype("float32")
    faiss.normalize_L2(q_emb)

    # Search the HNSW index but restrict to shortlist: to restrict, we'll fetch neighbors from index and filter
    # Approach: search global index with a larger k and then filter by shortlist set to preserve speed+recall.
    # Query global index for K = min(5000, len(df)) to get candidate pool, then intersect with shortlist.
    K_search = min(5000, len(df))
    D_all, I_all = index.search(q_emb, K_search)
    candidate_global = list(I_all[0])
    # filter to those in shortlist (preserve order by score)
    shortlist_set = set(shortlist_idx.tolist())
    filtered = [i for i in candidate_global if i in shortlist_set]
    # If filtered too small, fallback to shortlist_idx directly
    if len(filtered) < top_k:
        filtered = shortlist_idx.tolist()[:min(len(shortlist_idx), K_search)]

    # Build a small in-memory rerank index over filtered candidates using their embeddings
    cand_embs = embeddings[filtered].copy()
    faiss.normalize_L2(cand_embs)
    local_index = faiss.IndexFlatIP(cand_embs.shape[1])
    local_index.add(cand_embs)
    D, I = local_index.search(q_emb, top_k)

    results = []
    for rank, (local_idx, score) in enumerate(zip(I[0], D[0]), start=1):
        global_idx = int(filtered[int(local_idx)])
        results.append({
            "rank": rank,
            "index": global_idx,
            "title": df.iloc[global_idx]["title"],
            "abstract": df.iloc[global_idx]["abstract"],
            "score": float(score)
        })
    return results

# Example use (keep your upload logic or path selection):
# pdf_path = "uploaded_paper.pdf"
# res = recommend_papers_from_pdf(pdf_path, top_k=10, shortlist_k=1000)
# for r in res: print(r['rank'], r['score'], r['title'][:120])


In [19]:
# -----------------------------
# Final runner cell: use an already-uploaded PDF, run recommender, print & save results
# -----------------------------
import os
from pathlib import Path
import pandas as pd
from textwrap import shorten

# 1) Auto-detect uploaded PDF (common paths + the one you just uploaded)
CANDIDATE_PDFS = [
    "/content/ai-06-00183.pdf",        # your uploaded filename (note spelling)
    "/mnt/data/changed.pdf",
    "/content/chnaged.pdf",
    "/content/changed.pdf",
    "/content/uploaded_paper.pdf",
    "/content/changed.pdf"
]

pdf_path = None
for p in CANDIDATE_PDFS:
    if os.path.exists(p):
        pdf_path = p
        break

# If none found, also try listing /mnt/data and /content for any PDF
if pdf_path is None:
    for folder in ["/mnt/data", "/content"]:
        if os.path.exists(folder):
            for fname in os.listdir(folder):
                if fname.lower().endswith(".pdf"):
                    pdf_path = os.path.join(folder, fname)
                    break
        if pdf_path:
            break

if pdf_path is None:
    raise FileNotFoundError("No uploaded PDF found in usual locations. Please upload a PDF or set `pdf_path` manually.")

print("Using PDF:", pdf_path)

# 2) Choose which recommender function exists in this notebook
runner = None
if 'recommend_from_pdf' in globals():
    runner = globals()['recommend_from_pdf']
    runner_name = 'recommend_from_pdf'
elif 'recommend_papers_from_pdf' in globals():
    runner = globals()['recommend_papers_from_pdf']
    runner_name = 'recommend_papers_from_pdf'
elif 'recommend_from_abstract_text' in globals():
    # if only text-based runner exists, extract abstract then call it
    def _runner_wrapper(p, **kwargs):
        abs_text = extract_abstract_from_pdf(p)
        return recommend_from_abstract_text(abs_text, vectorizer=vectorizer, svd_model=svd_model,
                                            X_reduced=X_reduced, embeddings=embeddings, df=df,
                                            top_k=kwargs.get("top_k", 10), shortlist_k=kwargs.get("shortlist_k", 1000),
                                            model_name=kwargs.get("model_name", None))
    runner = _runner_wrapper
    runner_name = 'recommend_from_abstract_text (via wrapper)'
else:
    raise RuntimeError("No recommender function found. Ensure one of recommend_from_pdf / recommend_papers_from_pdf / recommend_from_abstract_text is defined in the notebook.")

print("Using recommender function:", runner_name)

# 3) Run recommender with safe defaults and print a preview
TOP_K = globals().get("TOP_K", 10)
SHORTLIST_K = globals().get("SHORTLIST_K", 1000)
MODEL_NAME = globals().get("EMBED_MODEL_NAME", globals().get("MODEL_NAME", "all-mpnet-base-v2"))

try:
    results = runner(pdf_path, top_k=TOP_K, shortlist_k=SHORTLIST_K) \
              if runner_name != 'recommend_from_abstract_text (via wrapper)' \
              else runner(pdf_path, top_k=TOP_K, shortlist_k=SHORTLIST_K)
except TypeError:
    # some functions may accept different arg names; try calling with no kwargs
    results = runner(pdf_path)

# 4) Pretty print top-K and save CSV
if not results:
    print("No results returned by the recommender.")
else:
    print(f"\nTop {min(TOP_K, len(results))} recommendations:")
    for r in results[:TOP_K]:
        score = float(r.get("score", 0.0)) if r.get("score") is not None else 0.0
        print(f"Rank {r.get('rank','?'):>2}  — idx {r.get('index','?'):>6}  — score {score:.4f}")
        print("Title   :", shorten(str(r.get('title','')), width=140, placeholder="..."))
        print("Abstract:", shorten(str(r.get('abstract','')), width=320, placeholder="..."))
        print("-"*80)

    # Save to CSV
    out_csv = Path("/content/recommendations.csv")
    try:
        pd.DataFrame(results).to_csv(out_csv, index=False)
        print("Saved recommendations CSV to:", out_csv)
    except Exception as e:
        print("Failed to save CSV:", e)

    # Display as a small DataFrame
    try:
        display(pd.read_csv(out_csv).head(20))
    except Exception:
        # fallback: show Python list converted to DataFrame
        display(pd.DataFrame(results).head(20))

# 5) Also show the extracted query abstract (so you can sanity-check the query)
try:
    if 'extract_abstract_from_pdf' in globals():
        query_abs = extract_abstract_from_pdf(pdf_path)
        print("\nExtracted query abstract (preview):")
        print(query_abs[:1000].strip())
    else:
        print("\nNo extract_abstract_from_pdf function found to show the query preview.")
except Exception as e:
    print("Could not extract/display query abstract:", e)


Using PDF: /content/ai-06-00183.pdf
Using recommender function: recommend_papers_from_pdf

Top 10 recommendations:
Rank  1  — idx  27061  — score 0.5848
Title   : Reinforcement learning in market games
Abstract: Financial markets investors are involved in many games -- they must interact with other agents to achieve their goals. Among them are those directly connected with their activity on markets but one cannot neglect other aspects that influence human decisions and their performance as investors. Distinguishing all...
--------------------------------------------------------------------------------
Rank  2  — idx   5759  — score 0.5500
Title   : Scalability and Optimisation of a Committee of Agents Using Genetic Algorithm
Abstract: A population of committees of agents that learn by using neural networks is implemented to simulate the stock market. Each committee of agents, which is regarded as a player in a game, is optimised by continually adapting the architecture of the agents us

,rank,index,title,abstract,score
0,1,27061,Reinforcement learning in market games,Financial markets investors are involved in ma...,0.584791
1,2,5759,Scalability and Optimisation of a Committee of...,A population of committees of agents that lear...,0.550011
2,3,46632,Risk-Seeking versus Risk-Avoiding Investments ...,We study the performance of various agent stra...,0.515383
3,4,45887,Trading Model with Pair Pattern Strategies,A simple trading model based on pair pattern s...,0.501171
4,5,26729,Adaptive Investment Strategies For Periodic En...,"In this paper, we present an adaptive investme...",0.483073
5,6,6099,A simple algorithm based on fluctuations to pl...,"In Biology, all motor enzymes operate on the s...",0.481924
6,7,39545,Hurst exponent and prediction based on weak-fo...,We empirically investigated the relationships ...,0.477805
7,8,22424,An empirical behavioral model of liquidity and...,We develop a behavioral model for liquidity an...,0.473125
8,9,21680,A Non Parametric Model for the Forecasting of ...,A neural net model for forecasting the prices ...,0.462822
9,10,8966,Multi-Agent Modeling Using Intelligent Agents ...,Game theory has many limitations implicit in i...,0.457113



Extracted query abstract (preview):
This paper presents a comprehensive deep reinforcement learning (DRL) framework for developing strategic trading models tailored to the Indian NIFTY 50 index, leveraging the temporal and nonlinear nature of financial markets. Three advanced DRL architec- tures deep Q-network (DQN), double deep Q-network (DDQN), and dueling double deep Q-network (Dueling DDQN) were implemented and empirically evaluated. Using a decade-long dataset of 15-min interval OHLC data enriched with technical indicators such as the exponential moving average (EMA), pivot points, and multiple supertrend configurations, the models were trained using prioritized experience replay, epsilon-greedy exploration strategies, and softmax sampling mechanisms. A test set comprising one year of unseen data (May 2024–April 2025) was used to assess generalization performance across key financial metrics, including Sharpe ratio, profit factor, win rate, and trade frequency. Each architecture 